In [1]:
# AI 만들기

# 데이터 불러오기 (학습 데이터 자료, 벡터 리스트, 단어-라벨링 사전) (pandas, pickle, custom.word_vectorize, numpy)
# x, t 분리
# 데이터 단어 라벨링
# Dataloader에 싣기 (torch, math) 
# 신경망 제작 (NN.py) #transformer 구조로 신경망 제작
# 신경망 학습

In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""),".."))
import custom

from IPython.display import display
import pandas
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import DataLoader

In [ ]:
# 벡터, 사전 불러오기
with open("../15 Movies_Korean/data/korean_vector_list.pkl", mode = "rb") as f :
    vector = pickle.load(f)
with open("../15 Movies_Korean/data/korean_word.pkl", mode = "rb") as f:
    word_dict = pickle.load(f)

vector = np.array(vector)
print(vector.shape)

(30187, 200)


In [4]:
#학습 데이터 불러오기

df = pandas.read_csv("mood.csv", encoding = "UTF8", sep = "\t")

display(df)


,text,word_len,mood_공포,mood_놀람,mood_분노,mood_슬픔,mood_중립,mood_행복,mood_혐오
0,언니 동생 으로 부르 는 것 이 맞 는 일 이 ㄴ가 요 ?,14,True,False,False,False,False,False,False
1,그냥 나 의 느낌 이 ㄹ 뿐 겠 지 ?,10,True,False,False,False,False,False,False
2,아직 너무 초기 이 라서 그런 거 이 죠 ?,10,True,False,False,False,False,False,False
3,유치원 버스 사고 나 엇 다던데,6,True,False,False,False,False,False,False
4,근데 원래 이런 거 맞 나 요,7,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
94189,얘기 ㄴ 다 끝나 었 냐 ? 원예 부,9,False,False,False,False,True,False,False
94190,예 . 그거 때문 에 부탁 이 있 . 는 . 데 요 .,14,False,False,False,False,True,False,False
94191,여자 숨기 어 달 라는 거 이 면 사절 이 다 .,12,False,False,False,False,True,False,False
94192,아무래도 안 되 나 요 ?,6,False,False,False,False,True,False,False


In [5]:
# x와 t로 나누고, x는 단어 라벨링, t는 argmax()
device = "cuda" if torch.cuda.is_available() else "cpu"

df = df[df['word_len'] > 0]

df_x = df['text'].values.tolist() #list로 변환
for i in range(len(df_x)) :
    df_x[i] = custom.word_vectorize(df_x[i], word_dict, 30)
tensor_x = torch.tensor(df_x, dtype = torch.long, device = device)

print(tensor_x.shape)


df_t = df.iloc[:,2:].values #numpy로 변환
df_t = np.argmax(df_t, axis = -1) #argmax 구하고
tensor_t = torch.tensor(df_t, dtype = torch.long, device = device)

print(tensor_t.shape)

print(tensor_x[94191])
print(tensor_t[94191])

torch.Size([94193, 30])
torch.Size([94193])
tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0, 11929,   136,    12,    45,   213,  6913],
       device='cuda:0')
tensor(4, device='cuda:0')


In [6]:
# Dataloader에 싣기

dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size=200, shuffle = True)
test_dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size=1000, shuffle = False)

In [7]:
# Ai 만들기 NN.py에서 만들었습니다

from NN import PositionEncoding
from NN import Encoder

In [8]:
# Ai 학습하기
tensor_vector = torch.tensor(vector, dtype = torch.float)
encoder = Encoder(tensor_vector, nhead=8, out_n=7, dim_feedforward=1024, dropout_p = 0.1, num_layers=2)
encoder.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(encoder.parameters(), lr = 0.0008)
epoch = 100
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    encoder.train() #dropout 켜주기
    for x, t in dataloader :
        # 순전파
        y = encoder(x)
        # 손실함수 계산
        loss = loss_function(y, t)
        loss_sum += loss.item()
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_sum /= len(dataloader)
    # 중간 acc 점검
    correct = 0
    total = 0
    encoder.eval()
    for x, t in test_dataloader :
        with torch.no_grad() :
            y = encoder(x)
            correct += (torch.argmax(y, dim = -1) == t).sum().item()
            total += len(x)
    acc = correct / total
    # earlystopper 구현
    if acc <= prev_acc + 0.001 :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
        torch.save(encoder, "encoder.pt")
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {cnt}")
    if cnt >= 5:
        print("train end")
        break
    

epoch 1 | loss 1.2471818987224765 | acc 0.588504453621819 | cnt 0
epoch 2 | loss 1.1602985210985641 | acc 0.6052891403819817 | cnt 0
epoch 3 | loss 1.13007237374656 | acc 0.6166594120582208 | cnt 0
epoch 4 | loss 1.1090072428851117 | acc 0.6236026031658404 | cnt 0
epoch 5 | loss 1.0955124544236816 | acc 0.6307581242767509 | cnt 0
epoch 6 | loss 1.081288174965326 | acc 0.6302591487690168 | cnt 1
epoch 7 | loss 1.0683009225851412 | acc 0.6417780514475598 | cnt 0
epoch 8 | loss 1.0581389129035286 | acc 0.6324567643030798 | cnt 1
epoch 9 | loss 1.0470161722723845 | acc 0.6450160839977493 | cnt 0
epoch 10 | loss 1.0353536602038487 | acc 0.6517469451020776 | cnt 0
epoch 11 | loss 1.0264790004985347 | acc 0.6535623666302167 | cnt 0
epoch 12 | loss 1.0176845027129868 | acc 0.6587432187105199 | cnt 0
epoch 13 | loss 1.0066958883512298 | acc 0.6613336447506715 | cnt 0
epoch 14 | loss 0.998049798285126 | acc 0.6688713598675061 | cnt 0
epoch 15 | loss 0.9882731189646792 | acc 0.6754960559701889 | 